<a href="https://colab.research.google.com/github/shlok-py/DeerHack2023-BroCode/blob/classification0/deerhackaudio_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 10,6
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB0
from keras.applications.densenet import DenseNet121
from tensorflow.keras.optimizers import Adam
import os

In [44]:
data_path = "/content/drive/MyDrive/Data/"


In [47]:
for i in os.listdir(data_path):
  print(i, "--->", len(os.listdir(os.path.join(data_path, i))))

bear ---> 100
Cheetah ---> 102
deer ---> 102
elephant ---> 100
Hippopotamus ---> 77
Monkey ---> 770
Spinny babbler ---> 104
tiger ---> 109
rhinoceros ---> 103
Human ---> 100
Fire ---> 94
Wolf ---> 105
Spotted Dove ---> 104


In [48]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

for i in os.listdir(data_path):
    impath = Path(os.path.join(data_path,i)).rglob("*.jpg")
    for img_p in impath:
        try:
            img = PIL.Image.open(img_p)
            print("success")
        except PIL.UnidentifiedImageError:
            print(img_p)
        except:
            print(img_p)

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [76]:
import os
import shutil
source_dir = '/content/drive/MyDrive/Data'
train_dir = '/content/drive/MyDrive/Data/train'

classes = ['Cheetah', 'deer', 'elephant', 'rhinoceros', 'tiger', 'Monkey', 'bear', 'Hippopotamus', 'Spotted Dove', 'Spinny babbler', 'Human', 'Fire', 'Wolf']
for class_name in classes:
  class_dir = os.path.join(source_dir, class_name)
  train_class_dir = os.path.join(train_dir, class_name)
  os.makedirs(train_class_dir, exist_ok=True)

  files = os.listdir(class_dir)[:80]
  for file in files:
    src_path = os.path.join(class_dir, file)
    dest_path = os.path.join(train_class_dir, file)
    shutil.copy(src_path, dest_path)

In [79]:
import os
import shutil

source_dir = '/content/drive/MyDrive/Data'
train_dir = '/content/drive/MyDrive/Splitted/train'
test_dir = '/content/drive/MyDrive/Splitted/test'

classes = ['Cheetah', 'Fire', 'Hippopotamus', 'Human', 'Monkey', 'Spinny babbler', 'Spotted Dove', 'Wolf', 'bear', 'deer','elephant','rhinoceros', 'tiger']

for class_name in classes:
    class_dir = os.path.join(source_dir, class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)
    
    files = os.listdir(class_dir)
    num_train = 80
    num_test = len(files) - num_train
    
    train_files = files[:num_train]
    test_files = files[num_train:]
    
    for file in train_files:
        src_path = os.path.join(class_dir, file)
        dest_path = os.path.join(train_class_dir, file)
        shutil.copy(src_path, dest_path)
        
    for file in test_files:
        src_path = os.path.join(class_dir, file)
        dest_path = os.path.join(test_class_dir, file)
        shutil.copy(src_path, dest_path)


In [86]:
for i in os.listdir(train_dir):
  print(i, "--->", len(os.listdir(os.path.join(train_dir, i))))

Cheetah ---> 80
Fire ---> 80
Hippopotamus ---> 76
Human ---> 80
Monkey ---> 80
Spinny babbler ---> 80
Spotted Dove ---> 80
Wolf ---> 80
bear ---> 80
deer ---> 80
elephant ---> 80
rhinoceros ---> 80
tiger ---> 80
forest ---> 78


In [87]:
for i in os.listdir(test_dir):
  print(i, "--->", len(os.listdir(os.path.join(test_dir, i))))

Cheetah ---> 22
Fire ---> 14
Hippopotamus ---> 19
Human ---> 20
Monkey ---> 20
Spinny babbler ---> 24
Spotted Dove ---> 24
Wolf ---> 25
bear ---> 20
deer ---> 22
elephant ---> 20
rhinoceros ---> 23
tiger ---> 29
forest ---> 20


In [93]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
)
train_generator = train_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/Splitted/train",
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 1112 images belonging to 14 classes.


In [94]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/Splitted/test",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 302 images belonging to 14 classes.


In [95]:

pretrained_model = tf.keras.applications.EfficientNetB0(include_top = False)
pretrained_model2 = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in pretrained_model2.layers:
  layer.trainable = False

inputs = layers.Input(shape = (224,224,3),name = "input_layer")
x = pretrained_model2(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(32, activation = 'relu')(x)
x = layers.Dense(14, activation = 'softmax')(x)
model = Model(inputs, x)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 32)                32800     
                                                                 
 dense_5 (Dense)             (None, 14)                462       
                                                                 
Total params: 7,070,766
Trainable params: 33,262
Non-trainable params: 7,037,504
____________________________________________

In [96]:

lr = 0.0001
optimizer = Adam(lr=lr)
model.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 32)                32800     
                                                                 
 dense_5 (Dense)             (None, 14)                462       
                                                                 
Total params: 7,070,766
Trainable params: 33,262
Non-trainable params: 7,037,504
____________________________________________

In [98]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

history = model.fit(
            train_generator,
            steps_per_epoch = 35,
            epochs=3,
            validation_data=validation_generator,
            validation_steps=200,
            verbose = 1
            )

Epoch 1/3
35/35 [==============================] - ETA: 0s - loss: 1.8253 - accuracy: 0.4757

35/35 [==============================] - 331s 9s/step - loss: 1.8253 - accuracy: 0.4757 - val_loss: 1.0348 - val_accuracy: 0.7881
Epoch 2/3
35/35 [==============================] - 236s 7s/step - loss: 0.6869 - accuracy: 0.8696
Epoch 3/3
35/35 [==============================] - 248s 7s/step - loss: 0.3742 - accuracy: 0.9218


In [99]:
model.save("model14c.h5")

In [100]:
loss, accuracy  = model.evaluate(validation_generator)
print("Test accuracy: ", accuracy)


10/10 [==============================] - 65s 6s/step - loss: 0.3187 - accuracy: 0.9238
Test accuracy:  0.9238410592079163
